In [35]:
import pyspark

#  I chose to use SparkSession instead of SparkContext because SparkSession 
#  eliminates the need to create multiple SparkContext 
#  (using more memory) maintaining all functionalities of SparkContext (built within Spark 3.0? SparkSession API)

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("practice").getOrCreate()

In [36]:

# I downloaded a sample dataset from Kaggle and was playing around with it
# Instead of using an RDD, I decided to use DataFrame, which is more efficient
# It also organzies things into a table, similar to a relational database (SQL) 
# might make it easier to paramterize data from SQL queries
df = spark.read.csv('cereal.csv', header=True, inferSchema=True)
df.show()

+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name| mfr|type|calories|protein| fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|null|   C|      70|      4|   1|   130| null|  5.0|     6|  null|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|null|   C|     120|      3|   5|  null|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|   K|   C|      70|      4|   1|   260| null| null|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|   K|   C|    null|   null|   0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|   R|   C|     110|      2|   2|  null|  1.0| 14.0|     8|  null|      25|    3|   1.0|0.75|34.384843|


In [37]:
# Shows the Column names of DataFrame

df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [38]:
df.drop("type")

DataFrame[name: string, mfr: string, calories: int, protein: int, fat: int, sodium: int, fiber: double, carbo: double, sugars: int, potass: int, vitamins: int, shelf: int, weight: double, cups: double, rating: double]

In [39]:
df.show()

+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name| mfr|type|calories|protein| fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|null|   C|      70|      4|   1|   130| null|  5.0|     6|  null|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|null|   C|     120|      3|   5|  null|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|   K|   C|      70|      4|   1|   260| null| null|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|   K|   C|    null|   null|   0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|   R|   C|     110|      2|   2|  null|  1.0| 14.0|     8|  null|      25|    3|   1.0|0.75|34.384843|


In [40]:
# .na.drop(how="any") drops any rows that have null values. There is also "all" which only
# removes rows if all values are null
df.na.drop(how="any").show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|         Apple Jacks|  K|   C|     110|      2|  0|   125|  1.0| 11.0|    14|    30|      25|    2|   1.0| 1.0|33.174094|
|             Basic 4|  G|   C|     130|      3|  2|   210|  2.0| 18.0|     8|     1|      25|    3|  1.33|0.75|37.038562|
|           Bran Chex|  R|   C|      90|      2|  1|   200|  4.0| 15.0|     6|   125|      25|    1|   1.0|0.67|49.120253|
|Cinnamon Toast Cr...|  G|   C|     120|      1|  3|   210|  0.0| 13.0|     9|    45|      25|    2|   1.0|0.75|19.823573|
|            Clusters|  G|   C|     110|      3|  2|   140|  2.0| 13.0|     7|   105|      25|    3|   1.0| 0.5|40.400208|
+---------------

In [41]:
# threshold
# the threshold removes rows that don't meet the "threshold." 
# In that, a row must have a n number of non-null values to not be removed
# If the amount of non-null values in a row meets or exceeds the threshold, it stays
# otherwise, it is removed

df.na.drop(how="any", thresh=15).show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|         Apple Jacks|  K|   C|     110|      2|  0|   125|  1.0| 11.0|    14|    30|      25|    2|   1.0| 1.0|33.174094|
|             Basic 4|  G|   C|     130|      3|  2|   210|  2.0| 18.0|     8|     1|      25|    3|  1.33|0.75|37.038562|
|           Bran Chex|  R|   C|      90|      2|  1|   200|  4.0| 15.0|     6|   125|      25|    1|   1.0|0.67|49.120253|
|         Bran Flakes|  P|   C|      90|      3|  0|  null|  5.0| 13.0|     5|   190|      25|    3|   1.0|0.67|53.313813|
|        Cap'n'Crunch|  Q|   C|     120|      1|  2|   220| null| 12.0|    12|    35|      25|    2|   1.0|0.75|18.042851|
|            Che

In [42]:
df.show()

+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name| mfr|type|calories|protein| fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|null|   C|      70|      4|   1|   130| null|  5.0|     6|  null|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|null|   C|     120|      3|   5|  null|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|   K|   C|      70|      4|   1|   260| null| null|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|   K|   C|    null|   null|   0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|   R|   C|     110|      2|   2|  null|  1.0| 14.0|     8|  null|      25|    3|   1.0|0.75|34.384843|


In [43]:
# the subset parameter only removes null values if they are in the specified subset
df.na.drop(subset=['sodium', "fiber"]).show()

+--------------------+---+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein| fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|All-Bran with Ext...|  K|   C|    null|   null|   0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|Apple Cinnamon Ch...|  G|   C|     110|   null|null|   180|  1.5| 10.5|    10|    70|      25|    1|   1.0|0.75|29.509541|
|         Apple Jacks|  K|   C|     110|      2|   0|   125|  1.0| 11.0|    14|    30|      25|    2|   1.0| 1.0|33.174094|
|             Basic 4|  G|   C|     130|      3|   2|   210|  2.0| 18.0|     8|     1|      25|    3|  1.33|0.75|37.038562|
|           Bran Chex|  R|   C|      90|      2|   1|   200|  4.0| 15.0|     6|   125|      25|    1|   1.0|0.67|49.120253|
|Cinnamo

In [44]:
# this is probably the function that is most similar to what i have to do?
# lit allows df to add columns in a constant manner

from pyspark.sql.functions import lit

# .filter searches the dataframe for null values in accordance with paramters and stores values in another df
# .isNull() finds null values in specified column
df_sodiumNull = df.filter(df.sodium.isNull())

#,WithColumn adds columns to existing dataframe
errorLog = df_sodiumNull.withColumn("Reason", lit("Sodium Null"))

In [45]:
errorLog.show()

+-----------------+----+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+-----------+
|             name| mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|     Reason|
+-----------------+----+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+-----------+
|100% Natural Bran|null|   C|     120|      3|  5|  null|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|Sodium Null|
|   Almond Delight|   R|   C|     110|      2|  2|  null|  1.0| 14.0|     8|  null|      25|    3|   1.0|0.75|34.384843|Sodium Null|
|      Bran Flakes|   P|   C|      90|      3|  0|  null|  5.0| 13.0|     5|   190|      25|    3|   1.0|0.67|53.313813|Sodium Null|
|         Cheerios|   G|   C|     110|      6|  2|  null|  2.0| 17.0|     1|   105|      25|    1|   1.0|1.25|50.764999|Sodium Null|
+-----------------+----+----+--------+-------+---+------+-----+-----+

In [46]:
errorLog = errorLog.union(df.filter(df.potass.isNull()).withColumn("Reason", lit("Pot Null")))
errorLog.show()

+-----------------+----+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+-----------+
|             name| mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|     Reason|
+-----------------+----+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+-----------+
|100% Natural Bran|null|   C|     120|      3|  5|  null|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|Sodium Null|
|   Almond Delight|   R|   C|     110|      2|  2|  null|  1.0| 14.0|     8|  null|      25|    3|   1.0|0.75|34.384843|Sodium Null|
|      Bran Flakes|   P|   C|      90|      3|  0|  null|  5.0| 13.0|     5|   190|      25|    3|   1.0|0.67|53.313813|Sodium Null|
|         Cheerios|   G|   C|     110|      6|  2|  null|  2.0| 17.0|     1|   105|      25|    1|   1.0|1.25|50.764999|Sodium Null|
|        100% Bran|null|   C|      70|      4|  1|   130| null|  5.0|

In [47]:
df.show()

+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name| mfr|type|calories|protein| fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+----+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|null|   C|      70|      4|   1|   130| null|  5.0|     6|  null|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|null|   C|     120|      3|   5|  null|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|   K|   C|      70|      4|   1|   260| null| null|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|   K|   C|    null|   null|   0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|   R|   C|     110|      2|   2|  null|  1.0| 14.0|     8|  null|      25|    3|   1.0|0.75|34.384843|


In [48]:
errorLog = errorLog.union(df.filter(df.vitamins < 25).withColumn("Reason", lit("Vitamins Value Incorrect")))
errorLog.show()

+-----------------+----+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+--------------------+
|             name| mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|              Reason|
+-----------------+----+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+--------------------+
|100% Natural Bran|null|   C|     120|      3|  5|  null|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|         Sodium Null|
|   Almond Delight|   R|   C|     110|      2|  2|  null|  1.0| 14.0|     8|  null|      25|    3|   1.0|0.75|34.384843|         Sodium Null|
|      Bran Flakes|   P|   C|      90|      3|  0|  null|  5.0| 13.0|     5|   190|      25|    3|   1.0|0.67|53.313813|         Sodium Null|
|         Cheerios|   G|   C|     110|      6|  2|  null|  2.0| 17.0|     1|   105|      25|    1|   1.0|1.25|50.764999|         Sodium Null|
|     